# Path e librerie
## Prima di modificare avvertire gli altri!

## Install

In [8]:
from google.colab import drive
drive_dir = 'drive' ; drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import

In [9]:
import tweepy
import numpy as np
from tweepy import Stream, OAuthHandler
from tweepy.streaming import StreamListener
from tqdm.notebook import tqdm
import pandas as pd
import io
import pprint
import os
import glob
import warnings
warnings.filterwarnings('ignore')

## Path

In [13]:
##########################
### ROOT
##########################

prj_root_dir = f'/content/{drive_dir}/MyDrive/nucleare/'
os.makedirs(prj_root_dir, exist_ok=True)

##########################
### SCRAPING
##########################

scrapingATP_root_dir = prj_root_dir + 'scrapingATP/'
os.makedirs(scrapingATP_root_dir, exist_ok=True)

##########################
### ETL
##########################

etl_root_dir = prj_root_dir + 'ETL/'
os.makedirs(etl_root_dir, exist_ok=True)

##########################
### DWH
##########################

dw_root_dir = prj_root_dir + 'DWH/'
os.makedirs(dw_root_dir, exist_ok=True)

##########################
### SENTIMENT
##########################

dw_root_dir = prj_root_dir + 'SENTIMENT/'
os.makedirs(dw_root_dir, exist_ok=True)

##########################
### WORDEMBEDDING
##########################

dw_root_dir = prj_root_dir + 'EMBEDDING/'
os.makedirs(dw_root_dir, exist_ok=True)

##########################
### OUTPUT
##########################

dw_root_dir = prj_root_dir + 'OUTPUT/'
os.makedirs(dw_root_dir, exist_ok=True)

# 1. Ingest
## Antonello

cartella di scraping: nulceare/scraping

## Codici Twitter

In [ ]:
consumer_key = 'MQXChIn29dCky3pOQyW890Uxi'
consumer_secret = 'u4n3UeFz58b4H6InWFhBDUPZsfJ8xdVS0wODdnfAjz6e6PUbCJ'
access_token = '262247836-oWkm6AZIw4K0Fldnt8VBXwuL9IbtNCZwVj73ODkI'
access_secret = 'GySGl7BGaYvmXUtCapu5pHwEDHD5tA3Qd4yCMIDPKm1Wx'
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

# consumer_key = 'Vzk6O5t967f1n4iz9iQcsQwhf'
# consumer_secret = 'Q8sncdY1j85mwNitOqb5Spcg6U6uuZ0E2KNHHZEYF2J4DdCyVi'
# access_token = '164224789-mFmU5YhessCNvubNojgfL605tbU0vh0NOna3HdQU'
# access_secret = 'Ic5LNOH2QqxziSq4eTQdJvbXo3noTcQDUzCKXgFcdPzpC'
# auth = OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_secret)
# api = tweepy.API(auth, wait_on_rate_limit=True)

# consumer_key = 'A4GRiWdUDgiGPckQSykzA'
# consumer_secret = 'nSyjpFDTr3eEmd8tFeBM1lwVrTNPLaHrcx0DicTEEuw'
# access_token = '1678747572-lbkg1jxpe5HypoTRSnRMeiVIe0RBMxhdjLJ0GJ6'
# access_secret = '1qVP0pHXV9HDgKhHDIy2HHqqnHFBm3sI5AR2ZDwZxlM'
# auth = OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_secret)
# api = tweepy.API(auth, wait_on_rate_limit=True)

time: 4.24 ms (started: 2021-11-15 21:12:05 +00:00)


## Scraping dello storico

In [ ]:
utente = 'antonello' #mettere il proprio nome
lang = 'all' #mettere la lingua
prog = '05' #mettere il progressivo

#mettere la query
query_00 = ('@janniksin' or 'NittoATPFinals' or 'TennisTakesOverTorino' or '#ATPFinals' or 'ATPFinals' or 'Djokovic' or 'Medvedev' or 'Zverev' or 'Tsitsipas' or 'Rublev' or 'Berrettini' or 'Hurkacz' or 'Ruud' or 'Sinner')

KeyboardInterrupt: ignored

time: 2.03 s (started: 2021-11-15 21:29:32 +00:00)


In [ ]:
searched_tweets = []
last_id = -1
max_tweets = 500_000
pbar = tqdm(total=max_tweets)

while len(searched_tweets) < max_tweets:
    count = max_tweets - len(searched_tweets)
    try:
        new_tweets = api.search(q=query_00, count=1000, max_id=str(last_id - 1), tweet_mode="extended", lang=language) # tweet_mode="extended" importante, altrimenti il testo del tweet viene troncato
        # If I've reached the end of the search then I'm done
        # immediately the cycle while
        if not new_tweets:
            break
        # added the data found to the list
        searched_tweets.extend(new_tweets)
        # tqdm
        pbar.update(len(new_tweets))
        # retrieve the last id found
        last_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        print(e)
        break

pbar.close()

In [ ]:
print('Totale Tweet')
len(searched_tweets)

In [ ]:
dict_ = {'id': [], 'user': [], 'date': [], 'text': [], 
         'favorite_count': [], 'hashtags': [], 
         'location': [], 'retweet': [], 'retweet_count': [], 'followers_count': [], 'in_reply_to_status_id':[], 'user_mentions':[]}

for status_j in searched_tweets[0:max_tweets]:
    status = status_j._json

    if 'retweeted_status' in status:
      if 'extended_tweet' in status['retweeted_status']:
        text = status['retweeted_status']['extended_tweet']['full_text']
        dict_['text'].append(text)
      else:
        text = status['retweeted_status']['text']
        dict_['text'].append(text)
    else:
      if 'extended_tweet' in status:
        text = status['extended_tweet']['full_text']
        dict_['text'].append(text)
      else:
        text = status['text']
        dict_['text'].append(text)
    
    dict_['id'].append(status['id'])
    dict_['user'].append(status['user']['screen_name'])
    dict_['hashtags'].append([hashtag['text'] for hashtag in status['entities']['hashtags']])
    dict_['user_mentions'].append([user_mention['screen_name'] for user_mention in status['entities']['user_mentions']])
    dict_['location'].append(status['user']['location'])
    dict_['followers_count'].append(status['user']['followers_count'])
    dict_['in_reply_to_status_id'].append(status['in_reply_to_status_id'])
    dict_['date'].append(status['created_at'])
    dict_['favorite_count'].append(status['favorite_count'])
    dict_['retweet_count'].append(status['retweet_count'])
    dict_['retweet'].append(status['retweeted'])

df = pd.DataFrame.from_dict(dict_, orient="columns")
df = df.set_index("id")
df.sort_values(by=['date'], inplace=True, ascending=True)
df.head(10)

KeyError: ignored

time: 27.3 ms (started: 2021-11-15 21:12:19 +00:00)


In [ ]:
df = pd.DataFrame.from_dict(dict_, orient="columns")
df = df.set_index("id")
df.sort_values(by=['date'], inplace=True, ascending=True)
df.head(10)

ValueError: ignored

time: 20.4 ms (started: 2021-11-15 21:13:03 +00:00)


In [ ]:
df.info()

In [ ]:
print(df.shape)

In [ ]:
df.to_csv(f'{scraping_root_dir}twitter_history_{utente}_{prog}_{lang}_{len(searched_tweets)}.csv')

## Scraping dello stream

In [ ]:
class MyStreamListener(tweepy.StreamListener):
    data = [] #qui ci metto i tweet che scarico man mano
    num = 0 #questo è un contatore settato a 0

    def store(self):
      dict_ = {'id': [], 'user': [], 'date': [], 'text': [],
         'favorite_count': [], 'hashtags': [], 
         'location': [], 'retweet': [], 'retweet_count': [], 'followers_count': [], 'in_reply_to_status_id':[], 'user_mentions':[]}
      
      for status_j in self.data:
          status = status_j._json

          if 'retweeted_status' in status:
            if 'extended_tweet' in status['retweeted_status']:
              text = status['retweeted_status']['extended_tweet']['full_text']
              dict_['text'].append(text)
            else:
              text = status['retweeted_status']['full_text']
              dict_['text'].append(text)
          else:
            if 'extended_tweet' in status:
              text = status['extended_tweet']['full_text']
              dict_['text'].append(text)
            else:
              text = status['full_text']
              dict_['text'].append(text)

          dict_['id'].append(status['id'])
          dict_['user'].append(status['user']['screen_name'])
          dict_['hashtags'].append([hashtag['text'] for hashtag in status['entities']['hashtags']])
          dict_['user_mentions'].append([user_mention['screen_name'] for user_mention in status['entities']['user_mentions']])
          dict_['location'].append(status['user']['location'])
          dict_['followers_count'].append(status['user']['followers_count'])
          dict_['in_reply_to_status_id'].append(status['in_reply_to_status_id'])
          dict_['date'].append(status['created_at'])
          dict_['full_text'].append(status['full_text'])
          dict_['favorite_count'].append(status['favorite_count'])
          dict_['retweet_count'].append(status['retweet_count'])
          dict_['retweet'].append(status['retweeted'])

      df = pd.DataFrame.from_dict(dict_, orient="columns")
      df = df.set_index("id")
      df.sort_values(by='favorite_count', inplace=True, ascending=False)
      df.to_csv(f'{scraping_root_dir}_twitter_stream_{utente}_{prog}_{lang}_' + str(self.num) + '.csv') #salvo in csv
      self.num = self.num + 1
      self.data = []


    def on_status(self, status):
        self.data.append(status)
        # to pandas
        # store su google drive
        if len(self.data) > 5000:
          self.store()

time: 37.9 ms (started: 2021-11-15 22:17:42 +00:00)


In [ ]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener, tweet_mode="extended")

time: 2.06 ms (started: 2021-11-15 22:17:45 +00:00)


In [ ]:
query = ['NittoATPFinals' or '#ATPFinals' or 'ATPFinals' or 'TennisTakesOverTorino' or 'Djokovic' or 'Medvedev' or 'Zverev' or 'Tsitsipas' or 'Rublev' or 'Berrettini' or 'Hurkacz' or 'Ruud' or 'Sinner']
myStream.filter(track=query)

KeyboardInterrupt: ignored

time: 4.37 s (started: 2021-11-15 22:17:51 +00:00)


In [ ]:
len(myStreamListener.data)

0

time: 4.37 ms (started: 2021-11-15 22:18:02 +00:00)


In [ ]:
dict_ = {'id': [], 'user': [], 'date': [], 'text': [], 
         'favorite_count': [], 'hashtags': [], 
         'location': [], 'retweet': [], 'retweet_count': [], 'followers_count': [], 'in_reply_to_status_id':[], 'user_mentions':[]}
#
for status_j in myStreamListener.data:
    status = status_j._json

    if 'retweeted_status' in status:
      if 'extended_tweet' in status['retweeted_status']:
        text = status['retweeted_status']['extended_tweet']['full_text']
        dict_['text'].append(text)
      else:
        text = status['retweeted_status']['text']
        dict_['text'].append(text)
    else:
      if 'extended_tweet' in status:
        text = status['extended_tweet']['full_text']
        dict_['text'].append(text)
      else:
        text = status['text']
        dict_['text'].append(text)

    dict_['id'].append(status['id'])
    dict_['user'].append(status['user']['screen_name'])
    dict_['hashtags'].append([hashtag['text'] for hashtag in status['entities']['hashtags']])
    dict_['user_mentions'].append([user_mention['screen_name'] for user_mention in status['entities']['user_mentions']])
    dict_['location'].append(status['user']['location'])
    dict_['followers_count'].append(status['user']['followers_count'])
    dict_['in_reply_to_status_id'].append(status['in_reply_to_status_id'])
    dict_['date'].append(status['created_at'])
    dict_['favorite_count'].append(status['favorite_count'])
    dict_['retweet_count'].append(status['retweet_count'])
    dict_['retweet'].append(status['retweeted'])

In [ ]:
df = pd.DataFrame.from_dict(dict_, orient="columns")
df = df.set_index("id")
df.sort_values(by='favorite_count', inplace=True, ascending=False)
df.head(10)

In [ ]:
df.to_csv(f'{scrapingATP_root_dir}twitter_stream_{utente}_{prog}_{lang}_{len(myStreamListener.data)}.csv') #scarico l'ultimo stream

In [14]:
#unisco tutti i csv

path = scrapingATP_root_dir

all_files = glob.glob(os.path.join(path, "twitter_*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged   = pd.concat(df_from_each_file, ignore_index=True)
df_merged.to_csv(f'{etl_root_dir}twitter_merged_from_scraping.csv', index=False, sep=',', encoding='utf-8')

In [15]:
df_merge = pd.read_csv(f'{etl_root_dir}twitter_merged_from_scraping.csv')
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34969 entries, 0 to 34968
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     34969 non-null  int64  
 1   user                   34969 non-null  object 
 2   date                   34969 non-null  object 
 3   text                   34969 non-null  object 
 4   favorite_count         34969 non-null  int64  
 5   hashtags               34969 non-null  object 
 6   location               20965 non-null  object 
 7   retweet                34969 non-null  bool   
 8   retweet_count          34969 non-null  int64  
 9   followers_count        34969 non-null  int64  
 10  in_reply_to_status_id  385 non-null    float64
 11  user_mentions          34969 non-null  object 
dtypes: bool(1), float64(1), int64(4), object(6)
memory usage: 3.0+ MB


In [17]:
df_merge.tail()

,id,user,date,text,favorite_count,hashtags,location,retweet,retweet_count,followers_count,in_reply_to_status_id,user_mentions
34964,1461093924110491649,chelstph,Wed Nov 17 22:08:31 +0000 2021,What a man 🥰\n\n@CasperRuud98 #NittoATPFinals ...,0,['NittoATPFinals'],NaN,False,0,73,NaN,"['TennisTV', 'CasperRuud98']"
34965,1461093952535384075,FLBZH35,Wed Nov 17 22:08:38 +0000 2021,Casper with the comeback 👏\n\n🇳🇴 @CasperRuud98...,0,['NittoATPFinals'],"Ille-et-Vilaine, Bretagne",False,0,26,NaN,"['atptour', 'CasperRuud98']"
34966,1461093959715930121,AleColumn99,Wed Nov 17 22:08:40 +0000 2021,💗 Bellissimo pensiero di @CasperRuud98 per gli...,0,"['tennis', 'NittoATPFinals']",Casatenovo,False,0,360,NaN,"['SuperTennisTv', 'CasperRuud98', 'steftsitsip..."
34967,1461093966556893187,pinkbassist,Wed Nov 17 22:08:42 +0000 2021,💗 Bellissimo pensiero di @CasperRuud98 per gli...,0,"['tennis', 'NittoATPFinals']",Toronto,False,0,703,NaN,"['SuperTennisTv', 'CasperRuud98', 'steftsitsip..."
34968,1461101661758361600,GrantMcKenzie10,Wed Nov 17 22:39:16 +0000 2021,The GREATEST defender of all time...\n\nNovak....,0,['NittoATPFinals'],"Elgin, Scotland",False,0,340,NaN,"['TennisTV', 'DjokerNole']"


# 2. ETL
**NATALIA**



1.   rimuovere #
2.   rimuovere id doppi
1.   rimuovere http
2.   rimuovere twitter con testo vuoto
1.   rimuovere @



file input: nucleare/ETL/twitter_merged_from_scraping.csv

cartella output: nucleare/DWH/

# 3. Sentiment
MARCO

# 4. BoW

TONE'

# 5. Word embeddings

# 6. Lime

# "SocialNetwork Analisys"